In [ ]:
#Python Code to send EMG to Flask, label and angle to Arduino 
import serial
import requests
from collections import deque
import time

ser = serial.Serial('COM8', 115200, timeout=1) 
server_url = "http://localhost:5000/predict"  

emg_buffer = deque(maxlen=10)  

try:
    while True:
        line = ser.readline().decode('utf-8').strip()
        if line and line.isdigit():
            
            normalized_emg = int(line) / 1023.0
            emg_buffer.append(normalized_emg)

            if len(emg_buffer) == 10:
                print("\nBuffer full. Sending to server...")
                try:
                    
                    response = requests.post(server_url, json={"emg": list(emg_buffer)})
                    
                    if response.status_code == 200:
                        data = response.json()
                        label = data.get("label", -1)
                        angle = data.get("angle", 0)
                        
                        if label != -1:
                            command = f"{label}:{angle}\n"
                            ser.write(command.encode())
                            print(f"Sent to Arduino: {command.strip()}")
                        else:
                            print("Error: Invalid label from server")
                    else:
                        print(f"Server error: {response.text}")
                
                except Exception as e:
                    print(f"Request failed: {e}")

except KeyboardInterrupt:
    ser.close()
    print("\nProgram terminated.")
